# Spatio-temporal coherence

Nous souhaitons nous assurer que les données décrivent un même phénomène physique sur une grille spatiale et temporelle cohérente, et cela, en maintenant une continuité et une exploitabilité pour l’analyse et la modélisation.

## 1. Cohérence temporelle

### 1.1 Vérifier la structure du temps

In [49]:
import xarray as xr

dsNOAA = xr.open_dataset("data/raw/merged/sstNOAA20102019.nc")
dsCOPERNICUS = xr.open_dataset("data/raw/merged/sstCOPERNICUS20102019.nc")

print(dsNOAA.time)
print(dsNOAA.time.encoding)

print("\n***************************************\n")

print(dsCOPERNICUS.time)
print(dsCOPERNICUS.time.encoding)

<xarray.DataArray 'time' (time: 3652)> Size: 29kB
array(['2010-01-01T00:00:00.000000000', '2010-01-02T00:00:00.000000000',
       '2010-01-03T00:00:00.000000000', ..., '2019-12-29T00:00:00.000000000',
       '2019-12-30T00:00:00.000000000', '2019-12-31T00:00:00.000000000'],
      shape=(3652,), dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 29kB 2010-01-01 2010-01-02 ... 2019-12-31
Attributes:
    long_name:     Time
    delta_t:       0000-00-01 00:00:00
    avg_period:    0000-00-01 00:00:00
    axis:          T
    actual_range:  [76701. 77065.]
{'dtype': dtype('float64'), 'zlib': False, 'szip': False, 'zstd': False, 'bzip2': False, 'blosc': False, 'shuffle': False, 'complevel': 0, 'fletcher32': False, 'contiguous': True, 'chunksizes': None, 'source': '/home/mkkuu/Documents/GitHub/m1Project_SciML/data/raw/merged/sstNOAA20102019.nc', 'original_shape': (3652,), '_FillValue': np.float64(nan), 'units': 'days since 1800-01-01', 'calendar': 'proleptic_gregorian'}


En comparant les axes temporels des deux jeux de données, on constate un léger décalage de borne temporelle. Copernicus incluant un jour supplémentaire au-delà de l'année 2019. Pour garantir la cohérence temporelle stricte entre les sources, nous allons simplement restreindre le jeu de donnée COPERNICUS à la période commune 2010-2019.

On note également une différence de standard de calendrier, NOAA utilisant le calendrier grégorien proleptique et COPERNICUS le grégorien simple. Cela ne fait aucune différence pour les années suivant 1582 d'après nos recherches. Aucun traitement ne sera réalisé.

In [50]:
print(dsNOAA.time.equals(dsCOPERNICUS.time)) # False, confirmant l'inégalité des bornes temporelles

dsCOPERNICUS = dsCOPERNICUS.sel(time=slice("2010-01-01", "2019-12-31")) # We apply slicing to ensure the same time range

print(dsNOAA.time.equals(dsCOPERNICUS.time)) # We check again after slicing

print(dsCOPERNICUS.time) # We print the time coordinate after slicing

False
False
<xarray.DataArray 'time' (time: 3653)> Size: 29kB
array(['2010-01-01T00:00:00.000000000', '2010-01-02T00:00:00.000000000',
       '2010-01-03T00:00:00.000000000', ..., '2019-12-29T00:00:00.000000000',
       '2019-12-30T00:00:00.000000000', '2019-12-31T00:00:00.000000000'],
      shape=(3653,), dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 29kB 2010-01-01 2010-01-02 ... 2019-12-31
Attributes:
    standard_name:  time
    long_name:      Time
    axis:           T


### 1.2 Continuité temporelle

On constate toujours une inégalité entre le nombre de points temporels dans les deux jeux de données. Il semblerait qu'il y ait toujours un point temporel supplémentaire (nous voulons avoir 3652 points correspondant à 8 années à 365 jours et 2 années bissextiles à 366 jours, soit 3652 jours). Explorons l'hypothèse d'un éventuel doublon en vérifiant le pas de temps.

In [51]:
import pandas as pd

dtNOAA = pd.to_datetime(dsNOAA.time.values).to_series().diff().dropna().unique()
dtCOPERNICUS  = pd.to_datetime(dsCOPERNICUS.time.values).to_series().diff().dropna().unique()

# For each dataset :
#   - convert time values to pandas datetime
#   - convert it to a pandas Series to manipulate it easily
#   - compute the difference between each time value and the previous one
#   - drop the NaT value resulting from the diff operation
#   - get the unique values of the resulting time differences

print(dtNOAA, dtCOPERNICUS) # Print delta times for both datasets


<TimedeltaArray>
['1 days']
Length: 1, dtype: timedelta64[ns] <TimedeltaArray>
['1 days', '0 days']
Length: 2, dtype: timedelta64[ns]


Résultant de notre test, on constate bel et bien la présence d'un doublon dans le jeu de donnée COPERNICUS. Pourquoi ? Car on remarque la présence de l'élément '0 days' dans la liste en retour de notre opération, signifiant qu'il existe 2 points SST pour un jour.

In [52]:
time = pd.to_datetime(dsCOPERNICUS.time.values) # We instanciate a pandas datetime object from the time values of the COPERNICUS dataset
dt = time.to_series().diff() # We convert it to a pandas Series and compute the time differences between each value and the previous one

duplicates = dt[dt == pd.Timedelta(0)] # We filter the delta times to get only the duplicates of 0 timedelta
print(duplicates) 

idx = duplicates.index # We get the index corresponding to the duplicate time values so it returns us the days where there are duplicates
print(idx)

2015-01-01   0 days
dtype: timedelta64[ns]
DatetimeIndex(['2015-01-01'], dtype='datetime64[ns]', freq=None)


Le jour présent 2 fois est donc le 01 janvier 2015. En effet, ayant télécharger manuellement depuis le site officiel sur les bornes 01/01/2010-01/01/2015 puis dans un second temps 01/01/2015-01/01/2020, cela a généré un doublon à la fusion des deux fichiers.

In [53]:
import numpy as np

index = np.unique(dsCOPERNICUS.time, return_index=True)[1] # We get the unique time values and their corresponding indices
dsCOPERNICUS = dsCOPERNICUS.isel(time=index) # We select only the unique time values using their indices

print(dsCOPERNICUS.time.equals(dsNOAA.time)) # We test the equality of the time bounds again after removing duplicates

True


La comparaison sur les axes temporelles des 2 jeux de données retourne True. Nous présumons ainsi la cohérence temporelle vérifiée et les fichiers bien alignés temporellement.

## 2. Cohérence spatiale

In [54]:
print(dsNOAA.dims)
print(dsNOAA.lat.min(), dsNOAA.lat.max())
print(dsNOAA.lon.min(), dsNOAA.lon.max())

print("\n***************************************\n")

print(dsCOPERNICUS.dims)
print(dsCOPERNICUS.latitude.min(), dsCOPERNICUS.latitude.max())
print(dsCOPERNICUS.longitude.min(), dsCOPERNICUS.longitude.max())


FrozenMappingWarningOnValuesAccess({'time': 3652, 'lat': 28, 'lon': 12})
<xarray.DataArray 'lat' ()> Size: 4B
array(-1.875, dtype=float32)
Attributes:
    long_name:      Latitude
    standard_name:  latitude
    units:          degrees_north
    actual_range:   [-89.875  89.875]
    axis:           Y <xarray.DataArray 'lat' ()> Size: 4B
array(4.875, dtype=float32)
Attributes:
    long_name:      Latitude
    standard_name:  latitude
    units:          degrees_north
    actual_range:   [-89.875  89.875]
    axis:           Y
<xarray.DataArray 'lon' ()> Size: 4B
array(48.125, dtype=float32)
Attributes:
    long_name:      Longitude
    standard_name:  longitude
    units:          degrees_east
    actual_range:   [1.25000e-01 3.59875e+02]
    axis:           X <xarray.DataArray 'lon' ()> Size: 4B
array(50.875, dtype=float32)
Attributes:
    long_name:      Longitude
    standard_name:  longitude
    units:          degrees_east
    actual_range:   [1.25000e-01 3.59875e+02]
    axis:   